****STDL PROJECT NOTEBOOK****

**Phase 1: predict gene expression levels of one specific gene over given input biopsy images**

Assign GPU device

In [1]:
# the next 2 lines are to allow debugging with CUDA !
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"  
print(f'cuda debugging allowed')

cuda debugging allowed


In [2]:
######
import torch
print(f'cuda device count: {torch.cuda.device_count()}')
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)
#Additional Info when using cuda
if device.type == 'cuda':
    print(f'device name: {torch.cuda.get_device_name(0)}')
    print(f'torch.cuda.device(0): {torch.cuda.device(0)}')
    print('Memory Usage:')
    print('Allocated:', round(torch.cuda.memory_allocated(0)/1024**3,1), 'GB')
    print('Cached:   ', round(torch.cuda.memory_cached(0)/1024**3,1), 'GB')
# NOTE: important !!!!!!
# clearing out the cache before beginning
torch.cuda.empty_cache()

cuda device count: 1
Using device: cuda
device name: GeForce RTX 2080
torch.cuda.device(0): <torch.cuda.device object at 0x7f756df8fbd0>
Memory Usage:
Allocated: 0.0 GB
Cached:    0.0 GB



Import project files

In [3]:
# create code to reimport module if i change it
%load_ext autoreload
%autoreload 2

# note: path to project is: /home/roy.rubin/STDLproject/
import loadAndPreProcess
import deepNetworkArchitechture

perform pre processing actions

In [4]:
print("\nHi! welcome to the program :)\n")


Hi! welcome to the program :)



In [5]:
path_to_images_dir = "/home/roy.rubin/STDLproject/spatialGeneExpressionData/images"
imageFolder = loadAndPreProcess.load_dataset_from_images_folder(path_to_images_dir)


----- entered function load_dataset_from_pictures_folder -----

----- finished function load_dataset_from_pictures_folder -----



In [6]:
path_to_images_dir = "/home/roy.rubin/STDLproject/spatialGeneExpressionData/images"
augmentedImageFolder = loadAndPreProcess.load_augmented_imageFolder_DS_from_images_folder(path_to_images_dir)


----- entered function load_dataset_from_pictures_folder -----

----- finished function load_dataset_from_pictures_folder -----



In [7]:
path_to_mtx_tsv_files_dir = "/home/roy.rubin/STDLproject/spatialGeneExpressionData"
matrix_dataframe, features_dataframe, barcodes_datafame = loadAndPreProcess.load_dataframes_from_mtx_and_tsv_new(path_to_mtx_tsv_files_dir)


----- entered function load_dataframes_from_mtx_and_tsv -----
started reading features.tsv
V  finished reading features.tsv
started reading barcodes.tsv
V  finished reading barcodes.tsv
started reading matrix.mtx. this might take some time ...
V  finished reading matrix.mtx

----- finished function load_dataframes_from_mtx_and_tsv -----



remove empty genes (i think there are only 3 of these ... need to verify)

In [8]:
%autoreload 2
matrix_dataframe, mapping_between_old_and_new_indices = loadAndPreProcess.cut_empty_genes(matrix_dataframe)  #TODO: uncomment later

cutting all genes (rows) that contain only zeros ...


In [9]:
# uncomment if wanted
# print(f'\note: this is the mapping_between_old_and_new_indices: \n{mapping_between_old_and_new_indices}')

# Create all of the custom datasets

In [10]:
gene_name = 'MKI67'
custom_DS_SingleValuePerImg = loadAndPreProcess.STDL_Dataset_SingleValuePerImg(imageFolder=imageFolder, 
                                                               matrix_dataframe=matrix_dataframe, 
                                                               features_dataframe=features_dataframe, 
                                                               barcodes_datafame=barcodes_datafame, 
                                                               chosen_gene_name=gene_name)
custom_DS_SingleValuePerImg_augmented = loadAndPreProcess.STDL_Dataset_SingleValuePerImg(imageFolder=augmentedImageFolder, 
                                                               matrix_dataframe=matrix_dataframe, 
                                                               features_dataframe=features_dataframe, 
                                                               barcodes_datafame=barcodes_datafame, 
                                                               chosen_gene_name=gene_name)


----- entering __init__ phase of  STDL_Dataset_SingleValuePerImg -----

----- finished __init__ phase of  STDL_Dataset_SingleValuePerImg -----


----- entering __init__ phase of  STDL_Dataset_SingleValuePerImg -----

----- finished __init__ phase of  STDL_Dataset_SingleValuePerImg -----



In [11]:
k = 10
custom_DS_KGenesWithHighestVariance = loadAndPreProcess.STDL_Dataset_KValuesPerImg_KGenesWithHighestVariance(imageFolder=imageFolder, 
                                                                           matrix_dataframe=matrix_dataframe, 
                                                                           features_dataframe=features_dataframe, 
                                                                           barcodes_datafame=barcodes_datafame, 
                                                                           num_of_dims_k=k)
custom_DS_KGenesWithHighestVariance_augmented = loadAndPreProcess.STDL_Dataset_KValuesPerImg_KGenesWithHighestVariance(imageFolder=augmentedImageFolder, 
                                                                           matrix_dataframe=matrix_dataframe, 
                                                                           features_dataframe=features_dataframe, 
                                                                           barcodes_datafame=barcodes_datafame, 
                                                                           num_of_dims_k=k)


----- entering __init__ phase of  STDL_Dataset_KValuesPerImg_KGenesWithHighestVariance -----
calculate variance of all columns from  matrix_dataframe - and choosing K genes with higest variance ...

----- finished __init__ phase of  STDL_Dataset_LatentTensor -----


----- entering __init__ phase of  STDL_Dataset_KValuesPerImg_KGenesWithHighestVariance -----
calculate variance of all columns from  matrix_dataframe - and choosing K genes with higest variance ...

----- finished __init__ phase of  STDL_Dataset_LatentTensor -----



In [12]:
k = 10
custom_DS_LatentTensor_NMF = loadAndPreProcess.STDL_Dataset_KValuesPerImg_LatentTensor_NMF(imageFolder=imageFolder, 
                                                                           matrix_dataframe=matrix_dataframe, 
                                                                           features_dataframe=features_dataframe, 
                                                                           barcodes_datafame=barcodes_datafame, 
                                                                           num_of_dims_k=k)
custom_DS_LatentTensor_NMF_augmented = loadAndPreProcess.STDL_Dataset_KValuesPerImg_LatentTensor_NMF(imageFolder=augmentedImageFolder, 
                                                                           matrix_dataframe=matrix_dataframe, 
                                                                           features_dataframe=features_dataframe, 
                                                                           barcodes_datafame=barcodes_datafame, 
                                                                           num_of_dims_k=k)


----- entering __init__ phase of  STDL_Dataset_KValuesPerImg_LatentTensor_NMF -----
performing NMF decomposition on main matrix dataframe ...

----- finished __init__ phase of  STDL_Dataset_LatentTensor -----


----- entering __init__ phase of  STDL_Dataset_KValuesPerImg_LatentTensor_NMF -----
performing NMF decomposition on main matrix dataframe ...

----- finished __init__ phase of  STDL_Dataset_LatentTensor -----



In [13]:
%autoreload 2

k = 10
custom_DS_LatentTensor_AE = loadAndPreProcess.STDL_Dataset_KValuesPerImg_LatentTensor_AutoEncoder(imageFolder=imageFolder, 
                                                                           matrix_dataframe=matrix_dataframe, 
                                                                           features_dataframe=features_dataframe, 
                                                                           barcodes_datafame=barcodes_datafame, 
                                                                           num_of_dims_k=k,
                                                                           device=device)
custom_DS_LatentTensor_AE_augmented = loadAndPreProcess.STDL_Dataset_KValuesPerImg_LatentTensor_AutoEncoder(imageFolder=augmentedImageFolder, 
                                                                           matrix_dataframe=matrix_dataframe, 
                                                                           features_dataframe=features_dataframe, 
                                                                           barcodes_datafame=barcodes_datafame, 
                                                                           num_of_dims_k=k,
                                                                           device=device)


----- entering __init__ phase of  STDL_Dataset_KValuesPerImg_LatentTensor_AutoEncoder -----
initializing the autoencoder (this might take a while) ...

----- entered function return_trained_AE_net -----
--delete-- verify inside STDL_Dataset_matrix_df_for_AE_init __init__: self.data.shape: (23073, 4992)
note - number of (hidden) linear layers is supposed to be 3

entered __init__ of AutoencoderNet
****** begin training ******

iteration 1 of 3 epochs
batch 4992 of 4992 batches
finished inner loop.

in this epoch: min loss 0.003137960797175765 max loss 2806.05908203125
               average loss 10.52984281306402

iteration 2 of 3 epochs
batch 4992 of 4992 batches
finished inner loop.

in this epoch: min loss 0.00298705929890275 max loss 2733.376953125
               average loss 7.655813598069542

iteration 3 of 3 epochs
batch 4992 of 4992 batches
finished inner loop.

in this epoch: min loss 0.003660012036561966 max loss 1233.6768798828125
               average loss 6.25686750954077


display a few sample images

In [14]:
# TODO: code to display some sample images
# 

check sizes of a pair (x,y) from all different datasets

In [15]:
# print(f'custom_DS_SingleValuePerImg (single gene):      x shape {custom_DS_SingleValuePerImg[0][0].shape}, y is an int')
# print(f'custom_DS_SingleValuePerImg_augmented:          x shape {custom_DS_SingleValuePerImg_augmented[0][0].shape}, y is an int')
# print("---------")
# print(f'custom_DS_KGenesWithHighestVariance:            x shape {custom_DS_KGenesWithHighestVariance[0][0].shape}, y shape {custom_DS_KGenesWithHighestVariance[0][1].shape}')
# print(f'custom_DS_KGenesWithHighestVariance_augmented:  x shape {custom_DS_KGenesWithHighestVariance_augmented[0][0].shape}, y shape {custom_DS_KGenesWithHighestVariance_augmented[0][1].shape}')
# print("---------")
# print(f'custom_DS_LatentTensor_NMF:                     x shape {custom_DS_LatentTensor_NMF[0][0].shape}, y shape {custom_DS_LatentTensor_NMF[0][1].shape}')
# print(f'custom_DS_LatentTensor_NMF_augmented:           x shape {custom_DS_LatentTensor_NMF_augmented[0][0].shape}, y shape {custom_DS_LatentTensor_NMF_augmented[0][1].shape}')
# print("---------")
# print(f'custom_DS_LatentTensor_AE:                      x shape {custom_DS_LatentTensor_AE[0][0].shape}, y shape {custom_DS_LatentTensor_AE[0][1].shape}')
# print(f'custom_DS_LatentTensor_AE_augmented:            x shape {custom_DS_LatentTensor_AE_augmented[0][0].shape}, y shape {custom_DS_LatentTensor_AE_augmented[0][1].shape}')


In [16]:
%autoreload 2
import executionModule

# define hyperparameters for the experiments
hyperparameters = dict()
hyperparameters['batch_size'] = 25
hyperparameters['max_alowed_number_of_batches'] = 99999
hyperparameters['precent_of_dataset_allocated_for_training'] = 0.8
hyperparameters['learning_rate'] = 1e-4
hyperparameters['num_of_epochs'] = 3
hyperparameters['channels'] = [32] 
hyperparameters['num_of_convolution_layers'] = len(hyperparameters['channels'])
hyperparameters['hidden_dims'] = [100]
hyperparameters['num_of_hidden_layers'] = len(hyperparameters['hidden_dims'])
hyperparameters['pool_every'] = 99999

Tests without having to restore the dimensions

In [17]:
executionModule.runExperimentWithModel_BasicConvNet(custom_DS_SingleValuePerImg, hyperparams=hyperparameters, device=device, dataset_name='single_gene')


----- entered function runExperimentWithModel_BasicConvNet -----
/ * \ ENTERED train_prediction_model / * \ 
recieved model ConvNet(
  (feature_extractor): Sequential(
    (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
  )
  (classifier): Sequential(
    (0): Linear(in_features=991232, out_features=100, bias=True)
    (1): ReLU()
    (2): Linear(in_features=100, out_features=1, bias=True)
  )
)
recieved loss_fn MSELoss()
recieved optimizer Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    eps: 1e-08
    lr: 0.0001
    weight_decay: 0
)
recieved num_of_epochs_wanted 3
recieved max_alowed_number_of_batches 99999
****** begin training ******

iteration 1 of 3 epochs
batch 122 of 122 batches
finished inner loop.

in this epoch: min loss 0.0001819812459871173 max loss 396.0445556640625
               average loss 3.610258524980397
               number of correct predictions: 2542 / 3050

iteration 2 of 3 epochs
batch 122 of 122 

In [18]:
executionModule.runExperimentWithModel_BasicConvNet(custom_DS_SingleValuePerImg_augmented, hyperparams=hyperparameters, device=device, dataset_name='single_gene')


----- entered function runExperimentWithModel_BasicConvNet -----
/ * \ ENTERED train_prediction_model / * \ 
recieved model ConvNet(
  (feature_extractor): Sequential(
    (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
  )
  (classifier): Sequential(
    (0): Linear(in_features=991232, out_features=100, bias=True)
    (1): ReLU()
    (2): Linear(in_features=100, out_features=1, bias=True)
  )
)
recieved loss_fn MSELoss()
recieved optimizer Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    eps: 1e-08
    lr: 0.0001
    weight_decay: 0
)
recieved num_of_epochs_wanted 3
recieved max_alowed_number_of_batches 99999
****** begin training ******

iteration 1 of 3 epochs
batch 488 of 488 batches
finished inner loop.

in this epoch: min loss 9.504583431407809e-05 max loss 306.73480224609375
               average loss 0.6752635390472039
               number of correct predictions: 11988 / 12200

iteration 2 of 3 epochs
batch 488 of 

In [19]:
executionModule.runExperimentWithModel_BasicConvNet(custom_DS_KGenesWithHighestVariance, hyperparams=hyperparameters, device=device, dataset_name='highest_variance')


----- entered function runExperimentWithModel_BasicConvNet -----
/ * \ ENTERED train_prediction_model / * \ 
recieved model ConvNet(
  (feature_extractor): Sequential(
    (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
  )
  (classifier): Sequential(
    (0): Linear(in_features=991232, out_features=100, bias=True)
    (1): ReLU()
    (2): Linear(in_features=100, out_features=10, bias=True)
  )
)
recieved loss_fn MSELoss()
recieved optimizer Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    eps: 1e-08
    lr: 0.0001
    weight_decay: 0
)
recieved num_of_epochs_wanted 3
recieved max_alowed_number_of_batches 99999
****** begin training ******

iteration 1 of 3 epochs
batch 122 of 122 batches
finished inner loop.

in this epoch: min loss 7084.02294921875 max loss 263481.53125
               average loss 28833.73088899206
               number of correct predictions: 127 / 3050

iteration 2 of 3 epochs
batch 122 of 122 batches
fi

In [20]:
executionModule.runExperimentWithModel_BasicConvNet(custom_DS_KGenesWithHighestVariance_augmented, hyperparams=hyperparameters, device=device, dataset_name='highest_variance')


----- entered function runExperimentWithModel_BasicConvNet -----
/ * \ ENTERED train_prediction_model / * \ 
recieved model ConvNet(
  (feature_extractor): Sequential(
    (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
  )
  (classifier): Sequential(
    (0): Linear(in_features=991232, out_features=100, bias=True)
    (1): ReLU()
    (2): Linear(in_features=100, out_features=10, bias=True)
  )
)
recieved loss_fn MSELoss()
recieved optimizer Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    eps: 1e-08
    lr: 0.0001
    weight_decay: 0
)
recieved num_of_epochs_wanted 3
recieved max_alowed_number_of_batches 99999
****** begin training ******

iteration 1 of 3 epochs
batch 488 of 488 batches
finished inner loop.

in this epoch: min loss 6970.97021484375 max loss 300416.125
               average loss 30164.106305231813
               number of correct predictions: 525 / 12200

iteration 2 of 3 epochs
batch 488 of 488 batches
fi

Tests that restore the dimensions after training (for now, only for 1 sample image)

In [21]:
executionModule.runExperimentWithModel_BasicConvNet(custom_DS_LatentTensor_NMF, hyperparams=hyperparameters, device=device, dataset_name='NMF')


----- entered function runExperimentWithModel_BasicConvNet -----
/ * \ ENTERED train_prediction_model / * \ 
recieved model ConvNet(
  (feature_extractor): Sequential(
    (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
  )
  (classifier): Sequential(
    (0): Linear(in_features=991232, out_features=100, bias=True)
    (1): ReLU()
    (2): Linear(in_features=100, out_features=10, bias=True)
  )
)
recieved loss_fn MSELoss()
recieved optimizer Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    eps: 1e-08
    lr: 0.0001
    weight_decay: 0
)
recieved num_of_epochs_wanted 3
recieved max_alowed_number_of_batches 99999
****** begin training ******

iteration 1 of 3 epochs
batch 122 of 122 batches
finished inner loop.

in this epoch: min loss 0.4740532338619232 max loss 28.61026382446289
               average loss 2.210003663037644
               number of correct predictions: 21736 / 3050

iteration 2 of 3 epochs
batch 122 of 122 b

In [22]:
# # temp to delete ?
# model = executionModule.runExperimentWithModel_BasicConvNet(custom_DS_LatentTensor_NMF, hyperparams=hyperparameters, device=device, dataset_name='NMF')


In [23]:
# # temp to delete ?
# executionModule.runDimensionalityRestorationExperiment_with_NMF_DS(dataset=custom_DS_LatentTensor_NMF, model=model, device=device)


In [24]:
executionModule.runExperimentWithModel_BasicConvNet(custom_DS_LatentTensor_NMF_augmented, hyperparams=hyperparameters, device=device, dataset_name='NMF')


----- entered function runExperimentWithModel_BasicConvNet -----
/ * \ ENTERED train_prediction_model / * \ 
recieved model ConvNet(
  (feature_extractor): Sequential(
    (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
  )
  (classifier): Sequential(
    (0): Linear(in_features=991232, out_features=100, bias=True)
    (1): ReLU()
    (2): Linear(in_features=100, out_features=10, bias=True)
  )
)
recieved loss_fn MSELoss()
recieved optimizer Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    eps: 1e-08
    lr: 0.0001
    weight_decay: 0
)
recieved num_of_epochs_wanted 3
recieved max_alowed_number_of_batches 99999
****** begin training ******

iteration 1 of 3 epochs
batch 488 of 488 batches
finished inner loop.

in this epoch: min loss 0.4157313406467438 max loss 41.11117935180664
               average loss 1.5334972513259435
               number of correct predictions: 72896 / 12200

iteration 2 of 3 epochs
batch 488 of 488

In [25]:
executionModule.runExperimentWithModel_BasicConvNet(custom_DS_LatentTensor_AE, hyperparams=hyperparameters, device=device, dataset_name='AE')


----- entered function runExperimentWithModel_BasicConvNet -----
/ * \ ENTERED train_prediction_model / * \ 
recieved model ConvNet(
  (feature_extractor): Sequential(
    (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
  )
  (classifier): Sequential(
    (0): Linear(in_features=991232, out_features=100, bias=True)
    (1): ReLU()
    (2): Linear(in_features=100, out_features=10, bias=True)
  )
)
recieved loss_fn MSELoss()
recieved optimizer Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    eps: 1e-08
    lr: 0.0001
    weight_decay: 0
)
recieved num_of_epochs_wanted 3
recieved max_alowed_number_of_batches 99999
****** begin training ******

iteration 1 of 3 epochs
batch 122 of 122 batches
finished inner loop.

in this epoch: min loss 1828.24853515625 max loss 39748.41015625
               average loss 5455.717826468046
               number of correct predictions: 235 / 3050

iteration 2 of 3 epochs
batch 122 of 122 batches


In [26]:
# # temp to delete ?
# model = executionModule.runExperimentWithModel_BasicConvNet(custom_DS_LatentTensor_AE, hyperparams=hyperparameters, device=device, dataset_name='AE')


In [27]:
# # temp to delete ?
# executionModule.runDimensionalityRestorationExperiment_with_AE_DS(dataset=custom_DS_LatentTensor_AE, model=model, device=device)


In [28]:
executionModule.runExperimentWithModel_BasicConvNet(custom_DS_LatentTensor_AE_augmented, hyperparams=hyperparameters, device=device, dataset_name='AE')


----- entered function runExperimentWithModel_BasicConvNet -----
/ * \ ENTERED train_prediction_model / * \ 
recieved model ConvNet(
  (feature_extractor): Sequential(
    (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
  )
  (classifier): Sequential(
    (0): Linear(in_features=991232, out_features=100, bias=True)
    (1): ReLU()
    (2): Linear(in_features=100, out_features=10, bias=True)
  )
)
recieved loss_fn MSELoss()
recieved optimizer Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    eps: 1e-08
    lr: 0.0001
    weight_decay: 0
)
recieved num_of_epochs_wanted 3
recieved max_alowed_number_of_batches 99999
****** begin training ******

iteration 1 of 3 epochs
batch 488 of 488 batches
finished inner loop.

in this epoch: min loss 1491.4306640625 max loss 57165.2421875
               average loss 5621.374486204054
               number of correct predictions: 940 / 12200

iteration 2 of 3 epochs
batch 488 of 488 batches
f